In [1]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import importlib
from IPython.display import HTML

from src.functions import *
from src.config import *
from src.allocating import*
from src.particles import *
from src.dimensionless_scaling_test import *
from src.calculating_observables import *
import src.functions
#importlib.reload(src.functions)

#####initialize simulation######
fig = plt.figure()

#allocate memory
pos=allocating_variables()                                                     
radii = size_of_particles(N_particles, 1, 0.1)
colours_particles = np.zeros(shape=(N_particles,N_steps),dtype='object')
colours_orientation = np.zeros(shape=(N_particles, N_steps),dtype='object')
orientation = np.zeros((N_particles, N_steps))

# initialize variables
pos[:,:,0]=cubic_latice(N_particles_x,N_particles_y)
colours_particles[:,:] = '-g' 
colours_orientation[:,:] = '-b'

scaling_force,scaling_torque,lambda_a,lambda_s,lambda_n,lambda_F_in,lambda_T_in=func_dimensionless_scaling(radii)



orientation[:,0] = orientation_at_0(N_particles) # orientation in radians!
print(orientation[:,0])
###perform simulation
for time_step in range(1,N_steps):
    print("step is",time_step)
    pos[:,:,time_step], orientation[:,time_step], colours_particles[:,time_step],colours_orientation[:,time_step],displacement = update_position_and_orientation(
        pos[:,:,time_step-1], 
        orientation[:,time_step-1],
        radii,
        colours_particles[:,time_step-1], 
        colours_orientation[:,time_step-1],time_size,scaling_force,scaling_torque,lambda_a,lambda_s,lambda_n,lambda_F_in,lambda_T_in)

# dataprocessing
orientation = np.radians(orientation)
radius = np.reshape(radii, (N_particles,1))
orientation_x = pos[:,0,:]+radius*np.cos(orientation)
orientation_y = pos[:,1,:]+radius*-np.sin(orientation)

print('data done')

# calculating observables
ACV=auto_correlation_velocity(pos,timestep=1)#what is the timestepsize?
MSD=mean_square_displacement(pos)
# diffusion stil needs testing
center_mass_in_time=center_of_mass(pos,radii,time_step)
Diffusion=func_diffusion(center_mass_in_time,time_size)

####create plot ####
lns = []
trans = plt.axes().transAxes
w = np.linspace(1,10,N_steps)
for steps in range (N_steps): # loop through timesteps
    x,y = particle(pos[:,0,steps], pos[:,1,steps], radii[:], N_particles)
    lns_timestep = []
    if (steps % 300) == 0:
        for i in range(N_particles): #loop through particles (want to get rid of 
                                     # this, how can we do this?)
            ln1, = plt.plot(x[:,i], y[:,i], colours_particles[i,steps], lw=2)

            ln2, = plt.plot([pos[i,0,steps],orientation_x[i,steps]],[pos[i,1,steps],orientation_y[i,steps]], colours_orientation[i,steps],lw=2)

            lns_timestep.append(ln1)
            lns_timestep.append(ln2)
            #lns_timestep.append(ln3)
        lns.append(lns_timestep)
        print("step animation is:",steps," out of ",N_steps)

plt.figure()

plt.xlabel('x')
plt.ylabel('y')
plt.grid()
plt.axes().set_aspect('equal')

ani = animation.ArtistAnimation(fig, lns, interval=100/time_size,blit=True)
#ani.save("test.html")
HTML(ani.to_html5_video())

#plt.show()



[-38.40484068  17.87071299  43.8716793  -11.90018335  -7.24085854
  37.5337617   -2.58321717 -43.41186795 -39.42906203  15.07938865
  26.36209383  -0.76134994  -0.61035145 -16.79547011 -29.02798823
 -32.0970197  -37.80975168  16.44317538 -20.73422496 -10.50837537
  43.55922777  30.38415508  37.69784048  10.59509687  -8.683978
 -31.85823434 -15.69248667  37.96111903  36.10707196  22.27375669
  13.65288391 -20.73125734  10.47040976  12.97724199  37.55195138
  16.61978973  10.95627344  16.89524861  11.12894219 -40.72613443
 -32.93344523   9.56165247 -26.50590592  22.46392036   7.32913835
  43.89067116  35.39057575  44.21994197   3.57495817  -3.49960802
  42.54135182  20.27913446   9.68074612  21.7908108   37.3550006
  16.90107096   1.77799005   4.84537116   9.52164927 -12.61728816
  17.44353889   1.17935432 -17.85745437   0.16085214  -5.35420404
 -14.16934438  13.14303677 -36.00509628   5.0521309  -14.22927428
  24.54661975  43.73640404 -10.52927773 -24.9668663  -38.68838266
  25.93076154

/home/student/project3/project_3/src/functions.py:191: RuntimeWarning: invalid value encountered in true_divide
  a+=np.where(np.logical_and(np.logical_and(d<=R1+R2 , np.abs(R1-R2)<=d),d>0.10),R1**2*np.arccos((d**2+R1**2-R2**2)/(2*d*R1))+R2**2*np.arccos((d**2-R1**2+R2**2)/(2*d*R2))-(1/2)*np.sqrt((-d+R1+R2)*(d+R1-R2)*(d-R1+R2)*(d+R1+R2)),0)
/home/student/project3/project_3/src/functions.py:191: RuntimeWarning: invalid value encountered in arccos
  a+=np.where(np.logical_and(np.logical_and(d<=R1+R2 , np.abs(R1-R2)<=d),d>0.10),R1**2*np.arccos((d**2+R1**2-R2**2)/(2*d*R1))+R2**2*np.arccos((d**2-R1**2+R2**2)/(2*d*R2))-(1/2)*np.sqrt((-d+R1+R2)*(d+R1-R2)*(d-R1+R2)*(d+R1+R2)),0)
/home/student/project3/project_3/src/functions.py:191: RuntimeWarning: invalid value encountered in sqrt
  a+=np.where(np.logical_and(np.logical_and(d<=R1+R2 , np.abs(R1-R2)<=d),d>0.10),R1**2*np.arccos((d**2+R1**2-R2**2)/(2*d*R1))+R2**2*np.arccos((d**2-R1**2+R2**2)/(2*d*R2))-(1/2)*np.sqrt((-d+R1+R2)*(d+R1-R2)*(d-R1+R2)*

step is 10
step is 11
step is 12
step is 13
step is 14
step is 15
step is 16
step is 17
step is 18
step is 19
step is 20
step is 21
step is 22
step is 23
step is 24
step is 25
step is 26
step is 27
step is 28
step is 29
step is 30
step is 31
step is 32
step is 33
step is 34
step is 35
step is 36
step is 37
step is 38
step is 39
step is 40
step is 41
step is 42
step is 43
step is 44
step is 45
step is 46
step is 47
step is 48
step is 49
step is 50
step is 51
step is 52
step is 53
step is 54
step is 55
step is 56
step is 57
step is 58
step is 59
step is 60
step is 61
step is 62
step is 63
step is 64
step is 65
step is 66
step is 67
step is 68
step is 69
step is 70
step is 71
step is 72
step is 73
step is 74
step is 75
step is 76
step is 77
step is 78
step is 79
step is 80
step is 81
step is 82
step is 83
step is 84
step is 85
step is 86
step is 87
step is 88
step is 89
step is 90
step is 91
step is 92
step is 93
step is 94
step is 95
step is 96
step is 97
step is 98
step is 99
step is 10

/opt/conda/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [7]:
deviation_0 = np.random.uniform(-0.785*180/np.pi, 0.785*180/np.pi, N_particles)
#deviation_0[deviation_0>0] = 360 - deviation_0[deviation_0>0]
print(abs(deviation_0))

[ 37.61247798   2.44430197  20.99159239   1.75793696  27.35468163
  37.19104393  24.96963501  27.03119709  37.31550799  14.93264061
  14.96432889  33.4001864   44.05470547  17.6505087   14.13263705
   0.19035793   5.515392     1.86946732  33.26376675  17.69033482
  40.98842394  30.32030185  16.15778703  39.74963013  10.68417687]


In [5]:
        u = np.array([orientation_x, orientation_y])
        v = np.array([init_orien_x[steps,i]+pos[i,0,steps], init_orien_y[steps,i]+pos[i,1,steps]])
        len_u = np.linalg.norm(u)
        len_v = np.linalg.norm(v)
        cosx = np.dot(u,v)/(len_u*len_v)
        delta_degrees = np.arccos(cosx)*180/np.pi
        if steps == 2:
            print(delta_degrees, i, 'delta_theta' )
        

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [2]:
print(8**4)

4096
